In [2]:
import lmdb
import cv2
import numpy as np
import sys
import shutil
import xml.etree.ElementTree as ET
from tqdm import tqdm
import os.path as osp

In [ ]:
print(lmdb.__version__)
lmdb_path = "test"
dir_path = "WIDER_test"

In [ ]:
# 递归获取目录（文件夹）下的所有文件路径
import os
def get_filepath(dir_path, list_name):
    for file in os.listdir(dir_path):  # 获取文件（夹）名
        file_path = os.path.join(dir_path, file)  # 将文件（夹）名补全为路径
        if os.path.isdir(file_path):  # 如果是文件夹，则递归
            get_filepath(file_path, list_name)
        else:
            list_name.append(file_path)  # 保存路径
    return list_name

res = []
res = get_filepath(dir_path,res)
# print(res)

In [ ]:
env = lmdb.open(lmdb_path, map_size=1073741824*4)  # size=1GB
cache = {}
for path in tqdm(res):
    with open(path,'rb') as f:
        bin = f.read()
    cache['{}'.format(path[:-4])] = bin
    
with env.begin(write=True) as txn:
    for k, v in tqdm(cache.items()):
        if isinstance(v, bytes):
            # 图片类型为bytes
            txn.put(k.encode(), v)
        else:
            # 标签类型为str, 转为bytes
            txn.put(k.encode(), v.encode())  # 编码

env.close()

In [ ]:
image_path = 'WIDER_train\\0--Parade\\0_Parade_marchingband_1_5.jpg'
label = '0_Parade_marchingband_1_5'

env = lmdb.open('lmdb_dir')
cache = {}  # 存储键值对

with open(image_path, 'rb') as f:
    # 读取图像文件的二进制格式数据
    image_bin = f.read()

# 用两个键值对表示一个数据样本
cache['image\_000'] = image_bin
cache['label\_000'] = label

with env.begin(write=True) as txn:
    for k, v in cache.items():
        if isinstance(v, bytes):
            # 图片类型为bytes
            txn.put(k.encode(), v)
        else:
            # 标签类型为str, 转为bytes
            txn.put(k.encode(), v.encode())  # 编码

env.close()

In [ ]:
env = lmdb.open('data\\WIDERFace\\WIDER_val\\Annotations')

with env.begin(write=False) as txn:
    # 获取图像数据
    image_bin = txn.get('WIDER_val\\Annotations\\0_Parade_marchingband_1_1004'.encode())
    # label = txn.get('WIDER_val\\0--Parade\\0_Parade_Parade_0_102'.encode()).decode()  # 解码
    print(image_bin)
    # 将二进制文件转为十进制文件（一维数组）
    # image_buf = np.frombuffer(image_bin, dtype=np.uint8)
    # # 将数据转换(解码)成图像格式
    # # cv2.IMREAD_GRAYSCALE为灰度图，cv2.IMREAD_COLOR为彩色图
    # img = cv2.imdecode(image_buf, cv2.IMREAD_COLOR)
    # cv2.imshow('image', img)
    # cv2.waitKey(0)

In [6]:
import lmdb
env = lmdb.open('data\WIDERFace\WIDER_train\\train')
txn=env.begin(write=True)
for key,value in txn.cursor():
    print(key)
txn.commit()
env.close()

b'WIDER_train\\0--Parade\\0_Parade_Parade_0_1014'
b'WIDER_train\\0--Parade\\0_Parade_Parade_0_1019'
b'WIDER_train\\0--Parade\\0_Parade_Parade_0_1040'
b'WIDER_train\\0--Parade\\0_Parade_Parade_0_1041'
b'WIDER_train\\0--Parade\\0_Parade_Parade_0_1042'
b'WIDER_train\\0--Parade\\0_Parade_Parade_0_106'
b'WIDER_train\\0--Parade\\0_Parade_Parade_0_109'
b'WIDER_train\\0--Parade\\0_Parade_Parade_0_11'
b'WIDER_train\\0--Parade\\0_Parade_Parade_0_110'
b'WIDER_train\\0--Parade\\0_Parade_Parade_0_123'
b'WIDER_train\\0--Parade\\0_Parade_Parade_0_126'
b'WIDER_train\\0--Parade\\0_Parade_Parade_0_127'
b'WIDER_train\\0--Parade\\0_Parade_Parade_0_129'
b'WIDER_train\\0--Parade\\0_Parade_Parade_0_132'
b'WIDER_train\\0--Parade\\0_Parade_Parade_0_136'
b'WIDER_train\\0--Parade\\0_Parade_Parade_0_138'
b'WIDER_train\\0--Parade\\0_Parade_Parade_0_140'
b'WIDER_train\\0--Parade\\0_Parade_Parade_0_142'
b'WIDER_train\\0--Parade\\0_Parade_Parade_0_145'
b'WIDER_train\\0--Parade\\0_Parade_Parade_0_148'
b'WIDER_train\\0

In [3]:
env = lmdb.open('data\WIDERFace\WIDER_train\Annotations')
data_infos = []
img_prefix='data/WIDERFace/WIDER_train/'

with env.begin(write=False) as txn:

    img_id = '0_Parade_marchingband_1_100'
    lmdb_xml_label = osp.join(img_prefix, 'Annotations',
                    f'{img_id}').replace('/','\\').replace('data\\WIDERFace\\','')
    # print(lmdb_xml_label)
    # print(list(lmdb_xml_label))
    bin = txn.get(lmdb_xml_label.encode())
    # print(bin)
    root = ET.fromstring(bin)
    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)
    folder = root.find('folder').text
    data_infos.append(
        dict(
            id=img_id,
            filename=osp.join(folder, img_id),#再从xml文件中得到图片的路径
            width=width,
            height=height))
    print(data_infos)

[{'id': '0_Parade_marchingband_1_100', 'filename': '0--Parade\\0_Parade_marchingband_1_100', 'width': 1024, 'height': 682}]


In [18]:
with open('tools\dist_trainval.sh') as f:
    line = f.readline()
    str = []
    while line:
        str.append(line)
        print(str)

